# check_spreadsheet_consistency

@author: David Clemens-Sewall

This notebook checks the consistency of the ASD spreadsheets in melinda-validation-Feb2025 regarding two points
 1. Do the position headers align between the albedo and I/R sections of the spreadsheet?
 2. Is there data in the final row (i.e., the 2500 nm)? If not that is an indication that the all data might be offset.

In [17]:
import os
import re
import copy
import glob
import pandas as pd
import numpy as np

In [4]:
# Data paths
mel_path = os.path.join('..', '..', 'melinda-validation-Feb2025')

In [5]:
# Get all day-sites that we have data for
in_dirs = [subdir for subdir in os.listdir(mel_path) if re.match(r"[0-9]{8}", subdir)]

In [88]:
# Check for position alignment and missing data

scan_pos_col_num = np.arange(41)*2 + 47
positions_forward = np.arange(41, dtype='int64')*5
positions_reverse = positions_forward[::-1]

in_dir = in_dirs[0]

for in_dir in in_dirs:
    filepaths = glob.glob(os.path.join(mel_path, in_dir, 'asd', '*line_asd-albedo*.a1.xlsx'))
    if len(filepaths) > 1:
        print('Multiple line albedo files')
        print(filepaths)
    elif len(filepaths) == 1:
        print(os.path.basename(filepaths[0]))
        if os.path.basename(filepaths[0])=='salvo_arm_line_asd-albedo_20240524.a1.xlsx':
            print('manually checked because of missing data')
            print('')
            continue
            
        ### Check position alignment
        albedo_positions = pd.read_excel(filepaths[0], header=None,
                                        usecols="D:AR", skiprows=3,
                                        nrows=1)
        scan_positions = pd.read_excel(filepaths[0], header=None,
                                       usecols=scan_pos_col_num, 
                                       skiprows=2, nrows=1)
        for i in np.arange(41):
            alb_pos = albedo_positions.at[0, 3+i]
            scan_pos = scan_positions.at[0, 47+2*i]
            if (alb_pos != scan_pos):
                print('Mismatch in ' + in_dir + ' alb_position ' + str(alb_pos))
        if np.array_equal(positions_forward, albedo_positions.T[0].values):
            print('forward')
        elif np.array_equal(positions_reverse, albedo_positions.T[0].values):
            print('reverse')
        else:
            print('check positions, neither forward nor reverse')
            
        ### Check missing data
        last_row = pd.read_excel(filepaths[0], header=None,
                                        usecols="D:AR", skiprows=2155,
                                        nrows=1)
        if last_row.isnull().any().any():
            print("missing values in last row")
        else:
            print('last row okay')
        
        ### Check that incident scan numbers align with notes documents
        alb_scan_numbers = pd.read_excel(filepaths[0], header=0,
                                                usecols=scan_pos_col_num, 
                                               skiprows=2, nrows=2)
        alb_scan_numbers = alb_scan_numbers.loc[1].T

        notespath = filepaths[0].replace('albedo', 'notes')
        notes_scan_numbers = pd.read_excel(notespath, header=0, skiprows=5,
                                          usecols=[0,1], index_col=0, dtype='str',
                                          names=['position', 'scannumber'])
        no_scannumber_errors = True
        for pos in alb_scan_numbers.index:
            notes_sn = str(notes_scan_numbers.at[pos, 'scannumber'])
            l_sn = len(notes_sn)
            alb_sn = alb_scan_numbers.at[pos]
            # Check if both are nan
            if (notes_sn=='nan') and (alb_sn != alb_sn):
                continue
            elif alb_sn[-4-l_sn:-4] != notes_sn:
                print("Notes and albedo do not match for " + in_dir)
                no_scannumber_errors = False
        if no_scannumber_errors:
            print('All scan numbers match between albedo and notes files')
    else:
        print('No line albedo file for ' + in_dir)
    print('')


salvo_ice_line_asd-albedo_20240417.a1.xlsx
forward
missing values in last row
All scan numbers match between albedo and notes files

salvo_arm_line_asd-albedo_20240419.a1.xlsx
forward
last row okay
All scan numbers match between albedo and notes files

salvo_beo_line_asd-albedo_20240420.a1.xlsx
forward
last row okay
All scan numbers match between albedo and notes files

salvo_ice_line_asd-albedo_20240421.a1.xlsx
forward
last row okay
All scan numbers match between albedo and notes files

salvo_beo_line_asd-albedo_20240523.a1.xlsx
forward
last row okay
All scan numbers match between albedo and notes files

salvo_arm_line_asd-albedo_20240524.a1.xlsx
manually checked because of missing data

salvo_arm_line_asd-albedo_20240525.a1.xlsx
forward
last row okay
All scan numbers match between albedo and notes files

salvo_ice_line_asd-albedo_20240526.a1.xlsx
forward
last row okay
All scan numbers match between albedo and notes files

salvo_arm_line_asd-albedo_20240527.a1.xlsx
forward
last row ok